# Topic Modelling : trouver des thèmes dans un corpus

Le topic modelling est un outil de machine learning non supervisé.

Il permet de trouver les thèmes majeurs dans un corpus de textes, de façon automatique (ou presque).

Pour appliquer le topic modelling, **gensim** est la librairie de référence
![](images/gensim.png)

IL faut d'abord l'installer, à l'aide de la commande suivante :
```
conda install -c anaconda gensim 
```

Il existe plusieurs modèle de Topic Modelling, les plus connus sont :
- LSA pour Latent Semantic Analysis, qui fonctionne un peu comme une analyse en composantes principales
- LDA pour Latent Dirichlet Allocation, qui fonctionne un peu comme un K-means

Nous allons ici nous focaliser sur la mise en oeuvre de la LDA, mais la méthodologie est la même pour la LSA : 
- Preprocessing des données textuelles : calcul du BOW et/ou TF-IDF
- Entrainement du modèle (e.g. LDA ou LSA)
- Interprétation des résultats et définition des thèmes majeurs
- Eventuelles itérations pour affiner les résultats

### Preprocessing des données

Nous allons réutiliser le dataset des revues amazon, et calculer le TF-IDF :

In [1]:
import pandas as pd

df = pd.read_csv('data/amazon_reviews.csv', sep='|')
df.head()

,review,class
0,great cd: my lovely pat has one of the great v...,1
1,one of the best game music soundtracks - for a...,1
2,batteries died within a year ...: i bought thi...,0
3,"works fine, but maha energy is better: check o...",1
4,great for the non-audiophile: reviewed quite a...,1


In [2]:
from sklearn.feature_extraction.text import TfidfVectorizer
from nltk.corpus import stopwords
from nltk.tokenize import word_tokenize

stops = stopwords.words('english')

def clean_data(quote):
    quote = quote.lower()
    tokens = word_tokenize(quote)
    token_punc = [t for t in tokens if t.isalpha()]
    token_stop = [t for t in token_punc if t not in stops]
    return token_stop

# On effectue le traitement en tokens pour la suite
df['tokens'] = df['review'].apply(clean_data)

In [3]:
# On calcule de TF-IDF sur les 1000 premières reviews
vectorizer = TfidfVectorizer(stop_words=stops, analyzer=lambda x: x)

tfidf = vectorizer.fit_transform(df['tokens'])

### Topic modelling

Nous allons ensuite passer à l'étape de Topic Modelling à l'aide de gensim.

La première étape est toujours la même : instancier le modèle de LDA qu'il faut importer au préalable de la façon suivante :
```python
from gensim.models import LdaModel
```

La classe `LdaModel` a la signature suivante :
```python
lda_model = gensim.models.ldamodel.LdaModel(corpus,
                                            id2word,
                                            num_topics, 
                                            random_state,
                                            chunksize,
                                            passes)
```

Avec :
- `Corpus` le TF-IDF ou BOW (calculé avec scikit-learn)
- `id2word` un dictionnaire avec les correspondances entre indices dans le corpus et mots (calculé par gensim)
- `num_topics` est le nombre désiré de thèmes
- `random_state` pour la reproductibilité (mettre toujours la même valeur)
- `chunksize` la taille du mini-batch (laisser par défaut)
- `passes` le nombre de fois que le modèle verra tout le corpus pour s'entrainer (1 par défaut, en général 5 ou 10 est très bien)


La seule information qui nous manque est donc le paramètre à donner en `id2word`.

On peut utiliser pour ça l'objet `Dictionary`, qui s'importe de la façon suivante 
```python
from gensim.corpora import Dictionary
```

Et qui s'applique directement sur les tokens (d'où la nécessité de le calculer au-dessus) :

In [4]:
from gensim.corpora import Dictionary
from nltk.tokenize import word_tokenize

id2word = Dictionary(df['tokens'])

Enfin un dernier détail avant de pouvoir utiliser le modèle, il faut convertir le TF-IDF de `scikit-learn` en un format propre à `gensim` à l'aide de la fonction `Sparse2Corpus` :

In [5]:
from gensim.matutils import Sparse2Corpus

tfidf_gensim = Sparse2Corpus(tfidf, documents_columns=False)

Finalement, il est possible d'instancier et d'entrainer notre modèle (le tout se faisant en même temps) :

In [6]:
from gensim.models import LdaModel

# On instancie et entraine un modèle qui trouvera 3 thèmes
lda = LdaModel(corpus=tfidf_gensim, id2word=id2word, num_topics=3, random_state=0, passes=5)

Il est maintenant possible d'afficher les thèmes majeurs, avec la méthode `.print_topics()` :

In [7]:
from pprint import pprint
pprint(lda.print_topics())

[(0,
  '0.001*"marvalettes" + 0.001*"hardwood" + 0.000*"portwe" + 0.000*"subtle" + '
  '0.000*"projects" + 0.000*"apes" + 0.000*"spins" + 0.000*"luce" + '
  '0.000*"epilator" + 0.000*"deviating"'),
 (1,
  '0.004*"marky" + 0.003*"flange" + 0.002*"rack" + 0.002*"remixed" + '
  '0.002*"meyer" + 0.002*"sansui" + 0.002*"meaningful" + 0.002*"sequel" + '
  '0.002*"lencioni" + 0.002*"unsullied"'),
 (2,
  '0.007*"breakthrough" + 0.005*"thumbprint" + 0.004*"lencioni" + '
  '0.004*"endorse" + 0.003*"meaningful" + 0.003*"rack" + 0.003*"supplemental" '
  '+ 0.002*"typewritter" + 0.002*"sansui" + 0.002*"breakdowns"')]


L'idée est ensuite d'interpréter ces résultats...

### Visualisation des résultats

Il existe une librairie dédiée de visualisation des résultats de LDA, nommée `pyLDAvis`.

Cette dernière doit au préalable être installée avec la commande suivante :
```
conda install -c conda-forge pyldavis 
```

L'utilisation est ensuite relativement simple, il suffit ensuite d'effectuer quelques imports et lignes de code :

In [ ]:
import pyLDAvis
import pyLDAvis.gensim

pyLDAvis.enable_notebook()
bow = [id2word.doc2bow(line) for line in df['tokens']]  # convert corpus to BoW format
vis = pyLDAvis.gensim.prepare(topic_model=lda, dictionary=id2word, corpus=bow)
vis

**Exercice :**

Appliquer le topic modelling au dataset `headlines.csv`, qui contient des titres de journaux.

In [9]:
df = pd.read_csv('data/headlines.csv')
df.head()

/opt/tljh/user/lib/python3.7/site-packages/ipykernel/ipkernel.py:287: DeprecationWarning: `should_run_async` will not call `transform_cell` automatically in the future. Please pass the result to `transformed_cell` argument and any exception that happen during thetransform in `preprocessing_exc_tuple` in IPython 7.17 and above.
  and should_run_async(code)


,headline_text
0,aba decides against community broadcasting lic...
1,act fire witnesses must be aware of defamation
2,a g calls for infrastructure protection summit
3,air nz staff in aust strike for pay rise
4,air nz strike to affect australian travellers
